In [ ]:
import sqlite3
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
import os


In [7]:
# Conexión a la base de datos SQLite
conn = sqlite3.connect('./data/metadatos.db')
cursor = conn.cursor()

# Creación de la tabla para almacenar metadatos
cursor.execute('''
CREATE TABLE IF NOT EXISTS documentos (
    id TEXT PRIMARY KEY,
    file_path TEXT,
    file TEXT,
    pagina INTEGER
)
''')
conn.commit()

In [8]:
# Inicializar el modelo de embeddings (usando LaBSE en este ejemplo)
embedding_model = SentenceTransformerEmbeddings(model_name="LaBSE")

/var/folders/1w/2mpv0jw11qj_v4tpnrdqv7t40000gn/T/ipykernel_1363/479594108.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="LaBSE")
/Users/mhurben/anaconda3/envs/proy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
faiss_index_dir = 'data/faiss_index'

# Si el directorio existe y tiene contenido, cargamos el índice FAISS; sino, lo inicializamos en None.
if os.path.exists(faiss_index_dir) and os.listdir(faiss_index_dir):
    faiss_vector_store = FAISS.load_local(faiss_index_dir, embedding_model)
    print("Índice FAISS cargado desde disco.")
else:
    faiss_vector_store = None
    print("No se encontró índice FAISS previo. Se creará uno nuevo.")

No se encontró índice FAISS previo. Se creará uno nuevo.


In [64]:
# ----------------------------
# Función para procesar un PDF y agregar sus fragmentos a FAISS y SQLite
# ----------------------------
def procesar_pdf(file_path):
    global faiss_vector_store

    # Cargar el documento PDF
    loader = PyPDFLoader(file_path)
    documentos = loader.load()

    # Dividir el contenido en fragmentos de 768 tokens (con 50 tokens de solapamiento)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=768, overlap_size=50)
    documentos_divididos = text_splitter.split_documents(documentos)

    for i, doc in enumerate(documentos_divididos):
        # Extraer la etiqueta de página o asignar un valor por defecto
        page = doc.metadata.get('page_label', f"pagina_{i}")

        # Obtener el nombre base del archivo
        file_name = file_path.split("/")[-1]
        nombre_sin_ext = file_name.split(".")[0]

        # Crear un ID único para cada fragmento
        doc_id = f'{nombre_sin_ext}-{page}-{i}'

        # Si el índice FAISS aún no se ha creado, lo inicializamos con el primer fragmento
        if faiss_vector_store is None:
            faiss_vector_store = FAISS.from_documents(
                [doc],
                embedding_model,
                ids=[doc_id]
            )
        else:
            # Agregar nuevos fragmentos al índice existente
            faiss_vector_store.add_texts(
                texts=[doc.page_content],
                ids=[doc_id]
            )

        # Almacenar metadatos en SQLite
        cursor.execute('''
        INSERT INTO documentos (id, file_path, file, pagina)
        VALUES (?, ?, ?, ?)
        ''', (doc_id, file_path, file_name, page))
        conn.commit()

In [11]:
# Lista de rutas de archivos PDF a procesar
rutas_pdfs = ['../01_data/2022202200133100001545.pdf', '../01_data/U0924996.pdf']

for ruta_pdf in rutas_pdfs:
    procesar_pdf(ruta_pdf)

In [12]:
# --------------------------------------------------------
# Persistir el índice FAISS en disco para su uso en futuras ejecuciones
# --------------------------------------------------------
if faiss_vector_store is not None:
    faiss_vector_store.save_local(faiss_index_dir)
    print("Índice FAISS guardado en disco.")

Índice FAISS guardado en disco.


In [13]:
def obtener_id_por_metadatos(file_path):
    cursor.execute(f"""
    SELECT * FROM documentos WHERE file_path = '{file_path}'
    """)
    #Todos los resultados
    resultados = cursor.fetchall()
    return resultados if resultados else None

In [14]:
doc_id = obtener_id_por_metadatos('../01_data/2022202200133100001545.pdf')
len(doc_id)

44

In [ ]:
# Busqueda de documentos

def buscar_documentos(query, k=5):
    vector_store = FAISS.load_local(faiss_index_dir, embedding_model, allow_dangerous_deserialization = True)

    # Obtener el embedding de la consulta
    query_embedding = embedding_model.embed_query(query)

    # Realizar búsqueda en el índice FAISS
    resultados = vector_store.similarity_search_with_score_by_vector(query_embedding, k=k) # DISTANCE L2

    return resultados

# Consulta de ejemplo
query = "¿Cuántos períodos se habilitan cada curso académico para solicitar concurrir a los actos extraordinarios de evaluación?"
resultados = buscar_documentos(query, 2)

In [63]:
resultados[0][0].page_content

'también en Consejo de Gobierno de 3 de febrero de 2022, señala que cada curso académico \nse habilitarán tres períodos para solicitar concurrir a los actos extraordinarios de evaluación, \nen los que el alumnado podrá solicitar indistintamente concurrir a actos extraordinarios de \nevaluación de asignaturas del primer semestre, segundo semestre o anuales. En dicha norma \nse señala que el calendario académico de cada curso establecerá las fechas de realización de'

In [ ]:
def eliminar_documento(file_name):
    cursor.execute(f"""
    SELECT id FROM documentos WHERE file = '{file_name}'
    """)
    conn.commit()

    doc_ids = cursor.fetchall()
    doc_ids = [id[0] for id in doc_ids]

    # MODIFICAR LA BD PARA PONER LA COLUMNA EN_USO = FALSE

    if doc_ids:
        faiss_vector_store.delete(doc_ids)
    
    print(f"Documento {file_name} eliminado del índice FAISS.")

eliminar_documento('2022202200133100001545.pdf')

Documento 2022202200133100001545.pdf eliminado del índice FAISS.
